# Dragon Knowledge Base - DKB V1
这是一个使用towhee 将文档知识库内容经过模型转换后存入Milvus数据库的例子
模型使用上选择了Hugging face的shibing624/text2vec-base-chinese，期望能达到中文搜索的效果

In [2]:
# 查看环境变量参数
%env ENV_FOR_DYNACONF=local
import os 
os.chdir('/Users/wangjialong/Documents/code/saic_project/global-vehicle-dragon/data-processing')

# 重新加载模块
from importlib import reload
from config.settings import Settings
import db.db_manager
import config.settings
import os
import numpy as np
from db.db_manager import DBConn
from db.model.dragon_knowledge_base import DragonKnowledgeBase
from towhee import pipe, ops
from pymilvus import MilvusClient, utility, connections
import pandas as pd
from sqlalchemy import func, create_engine
from db.db_manager import DBConn
from db.model.dragon_knowledge_base import DragonKnowledgeBase
from pymilvus import (
    connections,
    utility,
    FieldSchema, CollectionSchema, DataType,
    Collection,
)
from towhee import pipe, ops
import numpy as np
from towhee.datacollection import DataCollection
from db.milvus_manager import MilvusConn
from langchain.text_splitter import MarkdownTextSplitter


reload(config.settings)
reload(db.db_manager)
print(f"Debug: {bool(Settings.DEBUG)}")

# 参数
database_name = 'dkb_v1'
# 最大文本的长度
max_token = 512
# 查询分区数量
partition_num = 100
# my_model_name = 'shibing624/text2vec-base-chinese'
my_model_name = 'bert-base-chinese'


env: ENV_FOR_DYNACONF=local
settings loaded:  local
Debug: False


In [3]:
# 
connections.connect(host=Settings.MILVUS_URL, port=Settings.MILVUS_PORT)
client = MilvusClient(host=Settings.MILVUS_URL, port=Settings.MILVUS_PORT)
# 清理向量数据库
# client.drop_collection('dkb_v1')
print(utility.has_collection(collection_name = 'dkb_v1'))

if utility.has_collection(collection_name = 'dkb_v1'):
    print(f"describe: {client.describe_collection(collection_name = 'dkb_v1')}")
    print(f"num of entities: {client.num_entities(collection_name = 'dkb_v1')}")

2024-02-24 00:20:03,376 - 140704529881984 - milvus_client.py-milvus_client:553 - DEBUG: Created new connection using: 23dc2544039244e0bb5511656539d6be


True
describe: {'collection_name': 'dkb_v1', 'auto_id': True, 'num_shards': 1, 'description': 'reverse image search', 'fields': [{'field_id': 100, 'name': 'id', 'description': '', 'type': 5, 'params': {}, 'element_type': 0, 'auto_id': True, 'is_primary': True}, {'field_id': 101, 'name': 'name', 'description': '', 'type': 21, 'params': {'max_length': 255}, 'element_type': 0}, {'field_id': 102, 'name': 'embedding', 'description': '', 'type': 101, 'params': {'dim': 768}, 'element_type': 0}], 'aliases': [], 'collection_id': 447919517348397921, 'consistency_level': 2, 'properties': {}, 'num_partitions': 1}
num of entities: 0


In [7]:

milvus_conn = MilvusConn(database_name, 768)

# insert_pipe = (
#     pipe.input('id', 'name', 'content')
#         .map('content', 'vec', ops.text_embedding.dpr(model_name=my_model_name))
#         .map('vec', 'vec', lambda x: x / np.linalg.norm(x, axis=0))
#         .map(('name', 'vec'), 'insert_status', ops.ann_insert.milvus_client(host='127.0.0.1', port='19530', collection_name='dkb_v1'))
#         .output()
# )

pipe = pipeline('towhee/text-embedding')\
    .set_param(model_name='bert-base-chinese')\
    .to('towhee', 'l2_normalize', 'vector')


db_conn = DBConn()

count_query = f"SELECT count(1) FROM p_dragon.dragon_knowledge_base LIMIT {partition_num}"
total_num = db_conn.session.query(func.count(DragonKnowledgeBase.id)).scalar()
iteration_times = total_num // partition_num + (0 if total_num % partition_num == 0 else 1)
db_conn.close()

print(f"Records Count: {total_num} | iterations time: {iteration_times}")
for i in range(iteration_times):
    engine = create_engine(Settings.DB_URL)
    start_num = i * partition_num
    select_query = f"SELECT id, name, content as content FROM p_dragon.dragon_knowledge_base LIMIT {partition_num} OFFSET {start_num}"     
    with engine.connect() as connection, connection.begin():
        df = pd.read_sql_query(sql=select_query, con=connection.connection)
        for index, row in enumerate(df.itertuples(index=False), start=1):
            if row.id is None:
                break
            else:
                pass
                
            if len(row.content) > max_token:
                print(f"{row.id} 大于最大token长度，切割块")
                markdown_spilitter = MarkdownTextSplitter(chunk_size=max_token, chunk_overlap=0)
                docs = markdown_spilitter.create_documents([row.content])
                for split in docs:
                    data = {
                        'id': row.id,
                        'name': row.name,
                        'content': split.page_content
                    }
                    new_seg = pd.DataFrame(data, index=[0])
                    normalize_vector = pipe(split.page_content)
                    print(normalize_vector)
                    # insert_pipe(*new_seg)
            else:    
                print(f"处理数据 分片{i}|{index}")
                normalize_vector = pipe(row.page_content)
                print(normalize_vector)
                # insert_pipe(*row)
    engine.dispose()

milvus_conn.flush()
milvus_conn.stats()
print(f"all done")

2024-02-24 00:36:02,557 - 140704529881984 - milvus_client.py-milvus_client:553 - DEBUG: Created new connection using: c876f2d5ec3f4a6e8d4f6bdc5ad861bc


NameError: name 'pipeline' is not defined

# 模型效果

In [5]:
prompt_word = '物料响应确认'

ans_pipe = (
    pipe.input('question')
        .map('question', 'vec', ops.text_embedding.dpr(model_name=my_model_name))
        .map('vec', 'vec', lambda x: x / np.linalg.norm(x, axis=0))
        .map('vec', 'res', ops.ann_search.milvus_client(host=Settings.MILVUS_URL, port=Settings.MILVUS_PORT, collection_name='dkb_v1', limit=5))        
        .map('res', 'ids', lambda x: [i[0] for i in x])
        .output('question', 'ids')
)

ans = ans_pipe(prompt_word)
ans = DataCollection(ans)
db_conn = DBConn()
milvus_conn = MilvusConn(database_name, 768)
try:
    for ans_result in ans:
        for d_id in ans_result['ids']:
            
        #     query_result = db_conn.session.query(DragonKnowledgeBase)\
        #         .filter(DragonKnowledgeBase.name == d_id)\
        #         .first();
            # help(milvus_conn.client)
            print(f"{d_id}")
            # print(milvus_conn.get(d_id))
except Exception as e:
    print(f"{e}")
finally:
    db_conn.close()
# ans.show()

2024-02-22 18:29:48,532 - 140704529881984 - milvus_client.py-milvus_client:553 - DEBUG: Created new connection using: a2316b47ab614d1bb35f26e9b7b619f8


447890829204270755
447890829204270757
447890829204270759
447890829204270761
447890829204270753
